In [7]:
import os
import gc
import optuna
import numpy as np
from utils import *
import pandas as pd
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score
%run skills.ipynb
%run languages.ipynb
%run work_experiences.ipynb
%run education.ipynb

class config:
    path = '../../../datasets/garanti-bbva-data-camp/'
    sub_path = '../../../datasets/garanti-bbva-data-camp/submission.csv'
    skills_path = '../../../datasets/garanti-bbva-data-camp/skills.csv'
    languages_path = '../../../datasets/garanti-bbva-data-camp/languages.csv'
    education_path = '../../../datasets/garanti-bbva-data-camp/education.csv'
    exp_path = '../../../datasets/garanti-bbva-data-camp/work_experiences.csv'
    seed = 42
    n_folds = 6

In [8]:
train_df = pd.read_csv(os.path.join(config.path, 'train_users.csv'))
test_df = pd.read_csv(os.path.join(config.path, 'test_users.csv'))
sub = pd.read_csv(config.sub_path)

print(f'train_df shape: {train_df.shape}')
print(f'test_df shape: {test_df.shape}')
df = train_df.append(test_df).reset_index(drop = True)
df.head()

train_df shape: (53019, 4)
test_df shape: (13255, 3)


,user_id,industry,location,moved_after_2019
0,1301,Information Technology and Services,"Istanbul, Istanbul, Turkey",1.0
1,6950,Internet,"Istanbul, Istanbul, Turkey",0.0
2,4880,Online Media,Turkey,0.0
3,26046,Telecommunications,"Istanbul, Istanbul, Turkey",0.0
4,11005,Banking,"Istanbul, Turkey",0.0


In [10]:
def fix_location(dataframe: pd.DataFrame) -> pd.DataFrame:

    df_ = dataframe.copy()
    df_.loc[df_['location'].astype(str).str.contains('Kahraman Maras'), 'location'] = 'Kahramanmaras, Turkey'
    df_.loc[df_['location'].astype(str).str.contains('Şanliurfa'), 'location'] = 'Sanliurfa, Turkey'
    df_.loc[df_['location'].astype(str).str.contains('İçel'), 'location'] = 'Mersin, Turkey'
    df_.loc[df_['location'].astype(str).str.contains('Afyon'), 'location'] = 'Afyonkarahisar, Turkey'
    df_['location'] = df_['location'].apply(lambda x: str(x).replace('Türkiye', 'Turkey'))
    df_['location'] = df_['location'].apply(lambda x: x.upper().strip())
    df_['location'] = df_['location'].apply(lambda x: translation(str(x)))
    tr_cities = load_tr_cities()
    for city in tr_cities:
        df_['location'] = df_['location'].apply(lambda x: city if city in x else x)
        df_['based_on_tr'] = df_['location'].apply(lambda x: 1 if x in tr_cities or x == 'TURKEY' else 0)

    return df_

df = fix_location(df)

In [4]:
skills_df = load_skills(config.skills_path, 30)
lang_df = load_languages(config.languages_path)
edu_df = load_education(config.education_path, 12)
exp_df = load_work_experiences(config.exp_path)

df = df.merge(skills_df, on = ['user_id'], how = 'left')
df = df.merge(lang_df, on = ['user_id'], how = 'left')
df = df.merge(edu_df, on = ['user_id'], how = 'left')
df = df.merge(exp_df, on = ['user_id'], how = 'left')

print(df.shape)
df.head()

Frequency of top 20 skills before preprocess: 294433
Frequency of top 20 skills after preprocess: 318624


100%|██████████| 10/10 [00:02<00:00,  4.37it/s]


(66274, 78)


,user_id,industry,location,moved_after_2019,based_on_tr,skill_Java,skill_JavaScript,skill_SQL,skill_C#,skill_Software Development,...,degree_Master of Science,degree_Associate's degree,degree_Bachelor of Engineering,degree_Doctor of Philosophy,degree_Master of Business Administration,degree_Mühendislik Fakültesi Mezunu,total_experience,last_experience,unique_company,avg_days_to_quit
0,1301,Information Technology and Services,ISTANBUL,1.0,1,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1341.0,306.0,3.0,447.000000
1,6950,Internet,ISTANBUL,0.0,1,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2101.0,699.0,4.0,525.250000
2,4880,Online Media,TURKEY,0.0,1,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1826.0,610.0,4.0,456.500000
3,26046,Telecommunications,ISTANBUL,0.0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3409.0,1553.0,1.0,3409.000000
4,11005,Banking,ISTANBUL,0.0,1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3409.0,640.0,3.0,1136.333333


In [5]:
target = 'moved_after_2019'
cat_features = ['industry', 'location']
drop_features = ['user_id']

for caterogical_col in cat_features:
    df[caterogical_col] = df[caterogical_col].astype(str).astype("category")

train_set = df.loc[df[target].notnull()].drop(columns=drop_features, axis = 1)
test_set = df.loc[df[target].isnull()].drop(columns=drop_features, axis = 1)

print(f'train_set: {train_set.shape}')
print(f'test_set: {test_set.shape}')

train_set: (53019, 77)
test_set: (13255, 77)


In [11]:
def objective(trial):

    df_ = df.copy()
    skill_size = trial.suggest_int('skill_size', 10, 50)
    lang_size = trial.suggest_int('lang_size', 4, 20)
    study_size = trial.suggest_int('study_size', 4, 20)
    degree_size = trial.suggest_int('degree_size', 4, 20)

    skills_df = load_skills(config.skills_path, skill_size)
    lang_df = load_languages(config.languages_path, lang_size)
    edu_df = load_education(config.education_path, study_size, degree_size)
    exp_df = load_work_experiences(config.exp_path)

    df_ = df_.merge(skills_df, on = ['user_id'], how = 'left')
    df_ = df_.merge(lang_df, on = ['user_id'], how = 'left')
    df_ = df_.merge(edu_df, on = ['user_id'], how = 'left')
    df_ = df_.merge(exp_df, on = ['user_id'], how = 'left')

    target = 'moved_after_2019'
    cat_features = ['industry', 'location']
    drop_features = ['user_id']

    for caterogical_col in cat_features:
        df_[caterogical_col] = df_[caterogical_col].astype(str).astype("category")

    train_set = df_.loc[df_[target].notnull()].drop(columns=drop_features, axis = 1)
    test_set = df_.loc[df_[target].isnull()].drop(columns=drop_features, axis = 1)

    X = train_set.drop(columns = [target], axis = 1)
    y = train_set[target]

    params = {'one_hot_max_size': 179,
              'depth': 12,
              'boosting_type': 'Plain',
              'bootstrap_type': 'MVS'}

    params['verbose'] = False
    params['random_state'] = config.seed
    params['cat_features'] = cat_features
    params['eval_metric'] = 'Accuracy'
    params['allow_writing_files'] = False
    params["iterations"] = 5000

    model = CatBoostClassifier(**params)
    kf = KFold(n_splits=config.n_folds)

    scores = list()
    for idx, (train_ind, val_ind) in enumerate(kf.split(X, y)):
        print(f'fold: {idx+1}')
        model = CatBoostClassifier(**params)
        X_train = X.iloc[train_ind]
        y_train = y.iloc[train_ind]
        X_val = X.iloc[val_ind]
        y_val = y.iloc[val_ind]

        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=400,
            verbose=False,
        )

        val_pred = model.predict(X_val)
        scores.append(accuracy_score(y_val, val_pred))
        del X_train, y_train, X_val, y_val
        gc.collect()

    return np.mean(scores)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

[I 2023-02-07 10:27:53,659] A new study created in memory with name: no-name-a21c3264-b7ad-4c9b-aabd-dd4d54e81901


Frequency of top 20 skills before preprocess: 294433
Frequency of top 20 skills after preprocess: 318624


100%|██████████| 19/19 [00:04<00:00,  4.40it/s]


fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
fold: 6


[I 2023-02-07 10:45:39,541] Trial 0 finished with value: 0.7668381317464009 and parameters: {'skill_size': 16, 'lang_size': 7, 'study_size': 18, 'degree_size': 19}. Best is trial 0 with value: 0.7668381317464009.


...

[I 2023-02-07 15:47:45,381] Trial 18 finished with value: 0.7702143642417091 and parameters: {'skill_size': 44, 'lang_size': 11, 'study_size': 7, 'degree_size': 13}. Best is trial 11 with value: 0.7720249697252345.


Frequency of top 20 skills before preprocess: 294433
Frequency of top 20 skills after preprocess: 318624


100%|██████████| 8/8 [00:01<00:00,  4.38it/s]


fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
fold: 6


[I 2023-02-07 16:02:26,139] Trial 19 finished with value: 0.7678754519571375 and parameters: {'skill_size': 35, 'lang_size': 7, 'study_size': 4, 'degree_size': 8}. Best is trial 11 with value: 0.7720249697252345.


In [12]:
study.best_params

{'skill_size': 48, 'lang_size': 10, 'study_size': 9, 'degree_size': 11}

In [6]:
X = train_set.drop(columns=[target], axis=1)
y = train_set[target]

def objective(trial):

    params = {
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 120, 200),
        #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 6, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    params['eval_metric'] = 'Accuracy'
    params['cat_features'] = cat_features
    params['random_state'] = config.seed
    params['allow_writing_files'] = False
    params["iterations"] = 5000

    model = CatBoostClassifier(**params)
    kf = KFold(n_splits=config.n_folds)

    scores = list()
    for idx, (train_ind, val_ind) in enumerate(kf.split(X, y)):
        print(f'fold: {idx+1}')
        model = CatBoostClassifier(**params)
        X_train = X.iloc[train_ind]
        y_train = y.iloc[train_ind]
        X_val = X.iloc[val_ind]
        y_val = y.iloc[val_ind]

        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=400,
            verbose=False,
        )

        val_pred = model.predict(X_val)
        scores.append(accuracy_score(y_val, val_pred))
        del X_train, y_train, X_val, y_val
        gc.collect()

    return np.mean(scores)


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2023-02-07 02:58:19,452] A new study created in memory with name: no-name-da7bc7d7-99f6-4fd6-8beb-f3562d742998


fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
fold: 6


[I 2023-02-07 03:11:49,344] Trial 0 finished with value: 0.7687431849867695 and parameters: {'one_hot_max_size': 149, 'depth': 11, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.7687431849867695.


fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
fold: 6


[I 2023-02-07 03:34:29,279] Trial 1 finished with value: 0.7580489322240216 and parameters: {'one_hot_max_size': 124, 'depth': 12, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 9.565425252754782}. Best is trial 0 with value: 0.7687431849867695.


...

fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
fold: 6


[I 2023-02-07 08:02:47,342] Trial 5 finished with value: 0.7302475800379692 and parameters: {'one_hot_max_size': 197, 'depth': 12, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 5.880338042966729}. Best is trial 0 with value: 0.7687431849867695.


fold: 1
fold: 2
fold: 3
fold: 4
fold: 5


[W 2023-02-07 10:18:05,814] Trial 6 failed because of the following error: KeyboardInterrupt('')
Traceback (most recent call last):
  File "/Users/sercanyesiloz/opt/anaconda3/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/67/wq_xsymd3jvc5w2fx1ld_18h0000gn/T/ipykernel_42069/4198301420.py", line 37, in objective
    model.fit(
  File "/Users/sercanyesiloz/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py", line 5128, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/Users/sercanyesiloz/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py", line 2355, in _fit
    self._train(
  File "/Users/sercanyesiloz/opt/anaconda3/lib/python3.9/site-packages/catboost/core.py", line 1759, in _train
    self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_

KeyboardInterrupt: 